In [ ]:
from torch.utils.data import DataLoader
from dataset import AudioDataset
from config import TRAIN_DATASET, BATCH_SICE
from neural_net import get_model, train_model

In [ ]:
train_audio_dataset = AudioDataset(TRAIN_DATASET)
train_loader = DataLoader(train_audio_dataset, batch_size=BATCH_SICE, shuffle=True)

In [ ]:
model = get_model()
train_model(model, train_loader)